## Downloading Wico Dataset Uploaded On Kaggle

In [ ]:
!pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d manaspp/wico-graph
! unzip wico-graph

Streaming output truncated to the last 5000 lines.
  inflating: Non_Conspiracy_Graphs/2286/nodes.csv  
  inflating: Non_Conspiracy_Graphs/2286/plot.png  
  inflating: Non_Conspiracy_Graphs/2287/edges.txt  
  inflating: Non_Conspiracy_Graphs/2287/nodes.csv  
  inflating: Non_Conspiracy_Graphs/2287/plot.png  
  inflating: Non_Conspiracy_Graphs/2288/edges.txt  
  inflating: Non_Conspiracy_Graphs/2288/nodes.csv  
  inflating: Non_Conspiracy_Graphs/2288/plot.png  
  inflating: Non_Conspiracy_Graphs/2289/edges.txt  
  inflating: Non_Conspiracy_Graphs/2289/nodes.csv  
  inflating: Non_Conspiracy_Graphs/2289/plot.png  
  inflating: Non_Conspiracy_Graphs/229/edges.txt  
  inflating: Non_Conspiracy_Graphs/229/nodes.csv  
  inflating: Non_Conspiracy_Graphs/229/plot.png  
  inflating: Non_Conspiracy_Graphs/2290/edges.txt  
  inflating: Non_Conspiracy_Graphs/2290/nodes.csv  
  inflating: Non_Conspiracy_Graphs/2290/plot.png  
  inflating: Non_Conspiracy_Graphs/2291/edges.txt  
  inflating: Non_Consp

## Importing Necessary Libraries

In [ ]:
import random
import networkx as nx
import pandas as pd
import os
import statistics
import matplotlib.pyplot as plt
import numpy as np

def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r

### Extracting 5G Conspiracy Graphs

In [ ]:
temp = []

for i in range(1,413):
    temp.append(list_files(f'5G_Conspiracy_Graphs/{i}/'))

five_cons = []
for i in range(len(temp)):
  nodes = [k for k in temp[i] if 'nodes' in k][0]
  n = pd.read_csv(nodes,sep=",")
  s_node = [n['id'][k] for k in range(n.shape[0]) if n['time'][k] == 0][0]

  for j in range(len(temp[i])):
    if 'edges' in temp[i][j]:
      break
  f = pd.read_csv(temp[i][j],sep=" ",names=['source','target'])
  f['rumor_source'] = 0
  f.loc[f["source"] == s_node, "rumor_source"] = 1
  g = nx.from_pandas_edgelist(f,source='source', target='target',create_using = nx.DiGraph)
  five_cons.append((g,s_node)) 

### Extracting Other Conspiracy Graphs

In [ ]:
temp = []

for i in range(1,598):
    temp.append(list_files(f'Other_Graphs/{i}/'))

other_cons = []
for i in range(len(temp)):
  nodes = [k for k in temp[i] if 'nodes' in k][0]
  n = pd.read_csv(nodes,sep=",")
  s_node = [n['id'][k] for k in range(n.shape[0]) if n['time'][k] == 0][0]

  for j in range(len(temp[i])):
    if 'edges' in temp[i][j]:
      break
  f = pd.read_csv(temp[i][j],sep=" ",names=['source','target'])
  f['rumor_source'] = 0
  g = nx.from_pandas_edgelist(f,source='source', target='target',create_using = nx.DiGraph)
  other_cons.append((g,s_node)) 

### Extracting Non-Conspiracy Graphs

In [ ]:
temp = []

for i in range(1,2502):
    temp.append(list_files(f'Non_Conspiracy_Graphs/{i}/'))

non_cons = []
for i in range(len(temp)):
  nodes = [k for k in temp[i] if 'nodes' in k][0]
  n = pd.read_csv(nodes,sep=",")
  s_node = [n['id'][k] for k in range(n.shape[0]) if n['time'][k] == 0][0]

  for j in range(len(temp[i])):
    if 'edges' in temp[i][j]:
      break
  f = pd.read_csv(temp[i][j],sep=" ",names=['source','target'])
  f['rumor_source'] = 0
  g = nx.from_pandas_edgelist(f,source='source', target='target',create_using = nx.DiGraph)
  non_cons.append((g,s_node)) 

In [ ]:
from sklearn.metrics import accuracy_score
import math
import numpy as np

## Rumor Centrality

- The rumor centrality of a node is described as a number of
definite propagation paths starting from the origin node. `The node
having higher rumor centrality is the source of information propagation.`
<br/>
- For general network, the original network graph is converted into a
tree referred as a BFS tree using Breadth First Search (BFS) technique, in which any node in the graph assumed as source node is
considered as starting node for BFS.

- Given node u ∈ V in graph G =
(V,E), let T ⊂ G denote the breadth-first search tree of u with
respect to G. Then, the rumor centrality of u with respect to G
is obtained by computing it as per Equation 2.1 with respect to T, where $T_w^u$ is the size of the subtree of G that is rooted at w and
points away from u. 

- The estimated rumor source is the one with maximal rumor centrality
(ties broken uniformly at random).

<img src="http://drive.google.com/uc?export=view&id=189k6T_7btzanNAxHKI3CFobxV9alOlyx">

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  count = 1
  node_rc = []
  for j in i[0]:

    #We calculate the bfs tree for each node in the graph to calculate the rumor centrality for each of the node.

    t = nx.bfs_tree(i[0],source=j)

    '''Calculating all the tree sizes for the denominator of the formula.
       We take the dfs at every node to calculate the size of the graph starting at that node and pointing away from j.    
    '''

    T = []
    for k in list(t):
      T.append(len(list(nx.dfs_tree(t,source = k))))

    #Calculating Rumor Centrality as per formula 2.1

    rc = math.factorial(len(list(t)))/(np.prod(T))
    node_rc.append([j,rc])

    #Selecting the nodes that have maximum rumor centrality.
  try:
    max_rc = sorted(node_rc,key = lambda x : x[1],reverse = True)[0][1]
    calc_source.append([i[0] for i in node_rc if i[1] == max_rc])
  except:
    calc_source.append([])

    #Calculating the accuracy score of the metric to see if it could identify the actual source node in its predicted sources.
count = 0
for i in range(len(calc_source)):
  if real_source[i] in calc_source[i]:
    count += 1
print(count/len(real_source))

0.27184466019417475


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  count = 1
  node_rc = []
  for j in i[0]:
    t = nx.bfs_tree(i[0],source=j)
    T = []
    for k in list(t):
      T.append(len(list(nx.dfs_tree(t,source = k))))
    rc = math.factorial(len(list(t)))/(np.prod(T))
    node_rc.append([j,rc])
  try:
    max_rc = sorted(node_rc,key = lambda x : x[1],reverse = True)[0][1]
    calc_source.append([i[0] for i in node_rc if i[1] == max_rc])
  except:
    calc_source.append([])

count = 0
for i in range(len(calc_source)):
  if real_source[i] in calc_source[i]:
    count += 1
print(count/len(real_source))

0.21273031825795644


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  count = 1
  node_rc = []
  for j in i[0]:
    t = nx.bfs_tree(i[0],source=j)
    T = []
    for k in list(t):
      T.append(len(list(nx.dfs_tree(t,source = k))))
    rc = math.factorial(len(list(t)))/(np.prod(T))
    node_rc.append([j,rc])
  try:
    max_rc = sorted(node_rc,key = lambda x : x[1],reverse = True)[0][1]
    calc_source.append([i[0] for i in node_rc if i[1] == max_rc])
  except:
    calc_source.append([])

count = 0
for i in range(len(calc_source)):
  if real_source[i] in calc_source[i]:
    count += 1
print(count/len(real_source))

0.18072770891643342


## Closeness Centrality

- The node with the highest closeness centrality in the largest weakly connected network is taken as the possible rumor node.

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest clustering coefficient.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.7233009708737864


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  try:
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.7939698492462312


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  try:
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.8128748500599761


## Jordan Center

- For each vertex $V$ you have to find $V_m$ - the largest distance to any other vertex amongst the data returned from All pairs shortest path (Dijkstra). Then, the vertex with the smallest $V_m$ are the one in the graph center (Jordan Center).

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  jordan = []
  
  #Calculating all the shortest paths from a vertex to all other nodes and taking the maximum shortest path.
  max_cc = max(nx.weakly_connected_components(i[0]), key=len)
  cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))
  for j in nx.all_pairs_shortest_path(cc_l):
    jordan.append([j[0],max([len(k) for k in j[1].values()])])
  
  #Taking the node with the shortest maximum distance to other nodes.
  
  try:
    calc_source.append(sorted(jordan,key = lambda x: x[1])[0][0])
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.3422330097087379


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  jordan = []
  for j in nx.all_pairs_shortest_path(i[0]):
    jordan.append([j[0],max([len(k) for k in j[1].values()])])
  try:
    calc_source.append(sorted(jordan,key = lambda x: x[1])[0][0])
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.33835845896147404


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  jordan = []
  for j in nx.all_pairs_shortest_path(i[0]):
    jordan.append([j[0],max([len(k) for k in j[1].values()])])
  try:
    calc_source.append(sorted(jordan,key = lambda x: x[1])[0][0])
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5041983206717313


## Voronoi Cells (Modified)

- We first take the largest weakly connected component.
- Then we take 5 nodes with highest closeness centrality.
- Finally, we take the Voronoi Partition of each node in the top 2 nodes,
  and select the rumor node as the one with the largest partition size.

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  v_centres = []
  try:

    #calculating the largest weakly connected component to find closeness centrality.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))

    #Using the top 5 highest clustering coefficients as veronoi centres.

    for k, v in cc_l.items():
      v_centres.append([k,v])
    v_centres = sorted(v_centres,key = lambda x: x[1],reverse = True)
    v_centres = [k[0] for k in v_centres][:2]
  except:
    v_centres = []
  if v_centres != []:
    
    #Calculating the size of vornoi partitions and choosing the centre corresponding to the highest size of partition.
    
    v_cells = nx.voronoi_cells(i[0],v_centres)
    v_cells.pop('unreachable', None)
    calc_source.append(max(v_cells, key=lambda x: len(v_cells[x])))
  else:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5558252427184466


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  v_centres = []
  try:
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))
    for k, v in cc_l.items():
      v_centres.append([k,v])
    v_centres = sorted(v_centres,key = lambda x: x[1],reverse = True)
    v_centres = [k[0] for k in v_centres][:2]
  except:
    v_centres = []
  if v_centres != []:
    v_cells = nx.voronoi_cells(i[0],v_centres)
    v_cells.pop('unreachable', None)
    calc_source.append(max(v_cells, key=lambda x: len(v_cells[x])))
  else:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.52428810720268


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  v_centres = []
  try:
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.closeness_centrality(i[0].subgraph(list(max_cc)))
    for k, v in cc_l.items():
      v_centres.append([k,v])
    v_centres = sorted(v_centres,key = lambda x: x[1])
    v_centres = [k[0] for k in v_centres][:2]
  except:
    v_centres = []
  if v_centres != []:
    v_cells = nx.voronoi_cells(i[0],v_centres)
    v_cells.pop('unreachable', None)
    calc_source.append(max(v_cells, key=lambda x: len(v_cells[x])))
  else:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.001599360255897641


## Epidemic Center (Modified)

- We take the largest weakly connected component.
- Then we calculate the cycles basis and find the node that appears in most cycles. Ties are broken with degree of nodes.

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  g = i[0]
  try:

    #Taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(g), key=len)
    g = g.subgraph(list(max_cc))

    #Convert the component into undirectional and take the cycle basis.
    
    g = g.to_undirected()
    cycles = nx.cycle_basis(g)

    d_nodes = {}
    for c in cycles:
      for n in c:
        if n in d_nodes:
          d_nodes[n] += 1
        else:
          d_nodes[n] = 1
    l_nodes = []

    #We use degree to break ties.

    for d in d_nodes:
      l_nodes.append([d,d_nodes[d],g.degree(d)])
    l_nodes = sorted(l_nodes, key = lambda x : (x[1],x[2]), reverse = True)
    try:
      calc_source.append(l_nodes[0][0])
    except:
      calc_source.append(-1)
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.4077669902912621


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  g = i[0]
  try:
    max_cc = max(nx.weakly_connected_components(g), key=len)
    g = g.subgraph(list(max_cc))
    g = g.to_undirected()
    cycles = nx.cycle_basis(g)
    d_nodes = {}
    for c in cycles:
      for n in c:
        if n in d_nodes:
          d_nodes[n] += 1
        else:
          d_nodes[n] = 1
    l_nodes = []
    for d in d_nodes:
      l_nodes.append([d,d_nodes[d],g.degree(d)])
    l_nodes = sorted(l_nodes, key = lambda x : (x[1],x[2]), reverse = True)
    try:
      calc_source.append(l_nodes[0][0])
    except:
      calc_source.append(-1)
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5678391959798995


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  g = i[0]
  try:
    max_cc = max(nx.weakly_connected_components(g), key=len)
    g = g.subgraph(list(max_cc))
    g = g.to_undirected()
    cycles = nx.cycle_basis(g)
    d_nodes = {}
    for c in cycles:
      for n in c:
        if n in d_nodes:
          d_nodes[n] += 1
        else:
          d_nodes[n] = 1
    l_nodes = []
    for d in d_nodes:
      l_nodes.append([d,d_nodes[d],g.degree(d)])
    l_nodes = sorted(l_nodes, key = lambda x : (x[1],x[2]), reverse = True)
    try:
      calc_source.append(l_nodes[0][0])
    except:
      calc_source.append(-1)
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5133946421431428


## Betweenness Centrality

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.betweenness_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest betweenness centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.49271844660194175


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.betweenness_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest betweenness centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.48576214405360135


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.betweenness_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest betweenness centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.3678528588564574


## Eigenvector Centrality

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.eigenvector_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5970873786407767


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.eigenvector_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.6314907872696818


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  try:
    #taking the largest weakly connected component.

    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = nx.eigenvector_centrality(i[0].subgraph(list(max_cc)))

    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.5605757696921232


## DC + CC

- Degree Centrality + Closeness Centrality
- DC+CC is a combination of degree centrality and closeness centrality. In DC+CC, first DC
(degree centrality) selects nodes with highest degree in the infection graph G (V, E). 
- Then CC (closeness centrality) picks
the node with highest closeness centrality rank among the nodes
selected by DC. This node, finally, is considered as the source.
- Formally, the source estimated by DC+CC is given as:<br/>
  `DC+CC = $argmax_w$($argmax_u$($degree(u)$)), w∈u, u∈V` 

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    d_l = nx.in_degree_centrality(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with highest degree centrality.
    for k, v in d_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1],reverse = True)
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.7233009708737864


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    d_l = nx.in_degree_centrality(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with highest degree centrality.
    for k, v in d_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1],reverse = True)
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.7973199329983249


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    d_l = nx.degree_centrality(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with highest degree centrality.
    for k, v in d_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1],reverse = True)
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.8092762894842063


## EC + CC

- Eccentricity + Closeness Centrality
- EC+CC is a combination of eccentricity and closeness centrality. In EC+CC, first EC (eccentricity) selects the nodes with minimum eccentricity in the infection graph G (V, E). 
- Then CC (closeness centrality) picks
the node with highest closeness centrality rank among the nodes
selected by EC. This node, finally, is considered as the source.
- Formally, the source estimated by EC+CC is given as:<br/>
  `EC+CC = $argmax_w$($argmin_u$($eccentricity(u)$)), w∈u, u∈V`

In [ ]:
calc_source = []
real_source = [i[1] for i in five_cons]

for i in five_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    e_l = nx.eccentricity(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with lowest eccentricity.
    for k, v in e_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1])
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.06796116504854369


In [ ]:
calc_source = []
real_source = [i[1] for i in other_cons]

for i in other_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    e_l = nx.eccentricity(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with lowest eccentricity.
    for k, v in e_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1])
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.03015075376884422


In [ ]:
calc_source = []
real_source = [i[1] for i in non_cons]

for i in non_cons:
  try:
    #taking the largest weakly connected component.
    p_centres = []
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    e_l = nx.eccentricity(i[0].subgraph(list(max_cc)))

    #taking the 10 nodes with lowest eccentricity.
    for k, v in e_l.items():
      p_centres.append([k,v])
    p_centres = sorted(p_centres,key = lambda x: x[1])
    p_centres = [k[0] for k in p_centres][:10]

    #finding the node with maximum closeness centrality among the nodes selected by degree centrality
    max_cc = max(nx.weakly_connected_components(i[0]), key=len)
    cc_l = {}
    for j in p_centres:
      cc_l[j] = nx.closeness_centrality(i[0].subgraph(list(max_cc)),j)
    #getting the node with the largest eigenvector centrality.
    
    calc_source.append(max(cc_l, key=cc_l.get))
  except:
    calc_source.append(-1)

count = 0
for i in range(len(calc_source)):
  if real_source[i] == calc_source[i]:
    count += 1
print(count/len(real_source))

0.02958816473410636
